## Function for characteristics

In [ ]:
# FUNCTION: 
# listreal1, listimag1, maxchar = characteristics_function (Aeval, Beval)

# Matrices
# Aeval = np.asarray (Amat (ref[0], ref[1], ref[2], ref[3]))
# Beval = np.asarray (Bmat (ref[0], ref[1], ref[2], ref[3]))

def characteristics_function (Aeval, Beval):
    m1, _ = eig (Beval, b = Aeval, overwrite_a = True, overwrite_b = True, check_finite = True)

    listreal1 = []
    listimag1 = []

    for l1 in range (len (m1)):
        realpart1 = m1 [l1].real
        imagpart1 = m1 [l1].imag
        
        listimag1.append (imagpart1)
        listreal1.append (realpart1)

    maxchar = max (listreal1)

    return listreal1, listimag1, maxchar

## Function for amplification factors and eigenvectors

In [ ]:
# FUNCTION: 
# listreal2, listimag2, m2, vect2 = stability_function (Aeval, Beval, Ceval)

def stability_function (Aeval, Beval, Ceval):
    Acomplex   = Aeval.dot (1j)
    Bcomplex   = Beval.dot (1j)

    m2, vect2 = eig ( (-wavenumber_fourier*Bcomplex - Ceval), b =  -Acomplex, overwrite_a = True, overwrite_b = True, check_finite = True)

    listreal2 = []
    listimag2 = []

    for l2 in range (len (m2)):
        realpart2 = m2 [l2].real                
        imagpart2 = m2 [l2].imag
        
        listimag2.append (imagpart2)
        listreal2.append (realpart2)

    return listreal2, listimag2, m2, vect2



## Functions for stiffness

In [ ]:
def stiffness_function (Bm, visc, Cm, variable, dvariable):
        # Variational form
        R = - Bm + visc*Cm

        # Compute directional derivative about u in the direction of du (Jacobian)
        dF = derivative (R, variable, dvariable)
        dummy = (inner (Constant (0.0), v1) + inner (Constant (0.0), v2) + inner (Constant (0.0), v3) + inner (Constant (0.0), v4))*dx

        # Assemble stiffness form
        A_stiffness = PETScMatrix ()
        b_stiffness = PETScVector ()

        # assemble (dF, tensor = A_stiffness) # Alternative 1
        bcs = [] # Alternative 2
        A_stiffness, _ = assemble_system (dF, dummy, bcs = bcs, A_tensor = A_stiffness, b_tensor = b_stiffness) # Alternative 2

        # Matrix A1
        A1 = matrix(A_stiffness.array ())
        # Print condition number
        condnumber = LA.cond (A1)
        # print ("Condition number:", condnumber)

        # # Compute all eigenvalues of A x = \lambda x
        # eigensolver.solve () #(neigs)

        # #Exporting the real part of the eigenvectors and plotting eigenvalues
        # Real = []
        # Imag = []

        # for i in range (eigensolver.get_number_converged ()):
        #     r, c, rv, cv = eigensolver.get_eigenpair (i)

        #     # Real part of eigenvalues
        #     Real.append (r)
        #     Imag.append (c)

        Real, Imag = solver_stiffness (A_stiffness)

        return Real, Imag, A_stiffness